# NESS Short Course
## Introduction to the analysis of neural electrophysiology data
### Saturday, June 3, 2023
---
### Coherence

---
If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/NESS-Short-Course-2023.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/NESS-Short-Course-2023')

## Load the data and look at it.

In [ ]:
# Load modules we'll need.

from   scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the data.

# Note: if you're running on Google Colab, then load the .mat file like this:
# data = loadmat("/content/NESS-Short-Course-2023/NESS-Coherence-1.mat");

# Otherwise, use this code to load the mat file:
data = loadmat('NESS-Coherence-1.mat')  # Load the data,

E1 = data['E1']               # ... from the first electrode,
E2 = data['E2']               # ... and from the second electrode.
t = data['t'][0]              # Load the time axis
K = np.size(E1,0)             # Store number of trials.
N = np.size(E1,1)             # Store number of observations.
dt = t[1]-t[0]                # Store sampling interval.

In [ ]:
# Look at it.

f = plt.figure(figsize=(12, 4), dpi=80);
plt.plot(t,E1[0,:])
plt.plot(t,E2[0,:])
plt.xlabel('Time [s]')
plt.ylabel('EEG');

## Compute the trial-averaged (auto-)spectrum for each electrode.

In [ ]:
T = t[-1]                                        # Get the total time of the recording.
N = E1.shape[1]                                  # Determine the number of sample points per trial

# Compute the Fourier transform for each trial
xf = np.array([np.fft.rfft(x - x.mean()) for x in E1])  # ... in E1
yf = np.array([np.fft.rfft(y - y.mean()) for y in E2])  # ... and in E2

# Compute the spectra
Sxx = 2 * dt**2 / T * (xf * xf.conj())           # Spectrum of E1 trials
Syy = 2 * dt**2 / T * (yf * yf.conj())           # ... and E2 trials
Sxy = 2 * dt**2 / T * (xf * yf.conj())           # ... and the cross spectrum

f = np.fft.rfftfreq(N, dt)                       # Define the frequency axis

fig = plt.figure(figsize=(12, 4), dpi=80)
# Plot the average spectrum over trials in decibels vs frequency
plt.plot(f, 10 * np.log10(Sxx.mean(0).real), lw=3, label='Trial-averaged spectrum')  
# ... and the spectrum from the first trial for reference
plt.plot(f, 10 * np.log10(Sxx[0].real), 'k', label='Single-trial spectrum')  

plt.xlim([0, 50])                               # ... in select frequency range,
plt.xlabel('Frequency [Hz]')                     # ... with axes labelled.
plt.ylabel('Power [ mV^2/Hz]')
plt.title('Trial-averaged spectrum')
plt.legend();

## Compute the coherence between the two electrodes

In [ ]:
# Compute the spectra
Sxx = 2 * dt**2 / T * (xf * xf.conj()).mean(0)   # Spectrum of E1, averaged across trials
Syy = 2 * dt**2 / T * (yf * yf.conj()).mean(0)   # ... and E2, average across trials
Sxy = 2 * dt**2 / T * (xf * yf.conj()).mean(0)   # ... and the cross spectrum, averaged across trials

# Compute the coherence squared.
cohr_squared = (Sxy * Sxy.conj()) / (Sxx * Syy)

fig = plt.figure(figsize=(5, 4), dpi=80)
plt.plot(f, cohr_squared.real)                   # Plot coherence vs frequency,
plt.xlim([0, 50])                                # ... in a chosen frequency range,
plt.ylim([0, 1])                                 # ... with y-axis scaled,
plt.xlabel('Frequency [Hz]')                     # ... and with axes labeled.
plt.ylabel('Coherence')
plt.title('Coherence between two electrodes');